# ADK Tutorial

- 원본: 구글 ADK 공식 튜토리얼 (https://google.github.io/adk-docs/get-started/tutorial/)
- 번역: 신제용 (abel@even-my-dogs.com, https://abel.even-my-dogs.com/)

# 당신의 첫번째 지능형 에이전트 팀을 만드세요: ADK로 만드는 점점 발전하는 Weather Bot

이 튜토리얼은 [Quickstart example](https://google.github.io/adk-docs/get-started/quickstart/)에서 확장된 내용으로, [Agent Development Kit](https://google.github.io/adk-docs/get-started/)를 기반으로 합니다. 이제 여러분은 더 깊이 들어가서, 보다 정교한 **다중 에이전트 시스템**을 구축할 준비가 되었습니다.

우리는 단순한 기반에서 시작해 점진적으로 고급 기능을 추가하며 **Weather Bot 에이전트 팀**을 구축할 것입니다. 처음에는 날씨를 조회할 수 있는 단일 에이전트로 시작하고, 점점 다음과 같은 기능을 추가할 것입니다:

*   다양한 AI 모델(Gemini, GPT, Claude)의 활용
*   인사 및 작별 인사와 같은 별도의 작업을 위한 특화된 하위 에이전트 설계
*   에이전트 간 지능적인 위임 가능
*   지속적인 세션 상태를 통해 에이전트에 메모리 부여
*   콜백을 활용한 중요한 안전 가드레일 구현

**왜 Weather Bot 팀인가?**

이 사용 사례는 단순해 보일 수 있지만, 실제 환경에서 복잡한 에이전틱 애플리케이션을 구축하는 데 필수적인 ADK의 핵심 개념을 탐색하기에 실용적이고 공감 가는 캔버스를 제공합니다. 여러분은 상호작용을 구성하고, 상태를 관리하며, 안전을 보장하고, 여러 AI "두뇌"를 조율하는 방법을 배우게 됩니다.

**ADK가 뭐였지?**

ADK는 대규모 언어 모델(LLM)을 기반으로 한 애플리케이션 개발을 간소화하도록 설계된 Python 프레임워크입니다. 이는 추론, 계획, 도구 활용, 동적 사용자 상호작용, 팀 내 협업을 수행하는 에이전트를 구축하기 위한 강력한 빌딩 블록을 제공합니다.

**이 고급 튜토리얼을 통해 여러분은 다음을 마스터하게 됩니다:**

*   ✅ **도구 정의&사용:** 에이전트에 특정 능력(예: 데이터 가져오기)을 부여하는 Python 함수(`tools`)를 만들고, 이를 효과적으로 사용하는 방법을 에이전트에 지시하기
*   ✅ **유연한 멀티 LLM:** 다양한 선도 LLM(Gemini, GPT-4o, Claude Sonnet)을 LiteLLM 통합을 통해 구성하여, 작업에 가장 적합한 모델을 선택 가능하게 하기
*   ✅ **에이전트 위임&협동:** 특화된 하위 에이전트를 설계하고, 사용자 요청을 팀 내 가장 적절한 에이전트에게 자동으로 전달(`auto flow`)하도록 구성하기
*   ✅ **메모리를 위한 Session State:** `Session State`와 `ToolContext`를 활용하여, 에이전트가 대화 내내 정보를 기억하고 더욱 맥락 있는 상호작용을 제공할 수 있게 하기
*   ✅ **콜백을 이용한 Safety Guardrails:** `before_model_callback`과 `before_tool_callback`을 활용해 요청/도구 사용을 사전에 점검, 수정 또는 차단할 수 있도록 하여 애플리케이션의 안전성과 제어력 향상하기

**최종 목표 상태:**

이 튜토리얼을 완료하면, 여러분은 기능적인 다중 에이전트 Weather Bot 시스템을 구축하게 됩니다. 이 시스템은 단순히 날씨 정보를 제공할 뿐 아니라, 대화의 예절을 다루고, 마지막으로 조회한 도시를 기억하며, 정의된 안전 규칙 내에서 작동하게 됩니다. 이 모든 것이 ADK를 활용해 조율됩니다.

**사전 준비 사항:**

*   ✅ **Python 프로그래밍에 대한 확실한 이해**
*   ✅ **LLM, API, 에이전트 개념에 대한 친숙함**
*   ❗ **무엇보다도: ADK Quickstart 튜토리얼을 완료했거나, ADK의 기본 개념(Agent, Runner, SessionService, 기본 Tool 사용법)에 대한 이해가 있어야 함** — 이 튜토리얼은 그러한 개념 위에 구축됩니다.
*   ✅ **사용할 LLM의 API 키** (예: Gemini용 Google AI Studio, OpenAI Platform, Anthropic Console 등)

**에이전트 팀을 만들 준비가 되었나요? 시작해봅시다!**

In [ ]:
# @title Step 0: 설치
# ADK와 멀티 모델 서포트를 위한 LiteLLM을 설치합니다.

!pip install google-adk -q
!pip install litellm -q

print("Installation complete.")

In [ ]:
# @title 필요한 라이브러리를 불러옵니다.
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Libraries imported.")

In [ ]:
# @title API 키 설정하기 (실제 API키로 변경하세요!)

# --- IMPORTANT: 플레이스홀더를 실제 API키로 교체하세요. ---

# Gemini API Key (Get from Google AI Studio: https://aistudio.google.com/app/apikey)
os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY" # <--- 교체

# OpenAI API Key (Get from OpenAI Platform: https://platform.openai.com/api-keys)
os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_API_KEY' # <--- 교체

# Anthropic API Key (Get from Anthropic Console: https://console.anthropic.com/settings/keys)
os.environ['ANTHROPIC_API_KEY'] = 'YOUR_ANTHROPIC_API_KEY' # <--- 교체


# --- 키 확인 (선택적인 확인) ---
print("API Keys Set:")
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"OpenAI API Key set: {'Yes' if os.environ.get('OPENAI_API_KEY') and os.environ['OPENAI_API_KEY'] != 'YOUR_OPENAI_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"Anthropic API Key set: {'Yes' if os.environ.get('ANTHROPIC_API_KEY') and os.environ['ANTHROPIC_API_KEY'] != 'YOUR_ANTHROPIC_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")

# API 키를 직접 사용하도록 ADK 설정 (Vertex AI을 사용하지 않도록 설정)
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"


# @markdown **보안 참고사항:** API 키는 노트북에 직접 하드코딩하기보다는 Colab Secrets나 환경 변수(environment variables) 등을 사용하여 안전하게 관리하는 것이 모범 사례입니다. 위의 플레이스홀더 문자열을 실제 키로 교체하세요.

In [ ]:
# --- 더 쉽게 사용하기 위해 모델이름 상수 설정 ---

MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash-exp"

# Note: 구체적인 모델 이름은 달라질 수 있습니다. LiteLLM이나 제공자의 문서를 참고하세요.
MODEL_GPT_4O = "openai/gpt-4o"
MODEL_CLAUDE_SONNET = "anthropic/claude-3-5-sonnet-20240620"


print("\nEnvironment configured.")

---

## Step 1: 당신의 첫번째 에이전트 \- Basic Weather Lookup

이제 Weather Bot의 기본 구성 요소를 만들어보겠습니다: 특정 작업 — 날씨 정보를 조회하는 — 을 수행할 수 있는 단일 에이전트를 구축하는 것입니다. 이를 위해 두 가지 핵심 요소를 만들어야 합니다:

1. **도구(Tool):** 에이전트에게 날씨 데이터를 가져오는 *능력*을 부여하는 Python 함수  
2. **에이전트:** 사용자의 요청을 이해하고, 자신이 weather tool을 가지고 있음을 인식하며, 언제 어떻게 이를 사용할지 결정하는 AI "두뇌"

---

**1\. 도구 정의하기 (`get_weather`)**

ADK에서 **도구**는 에이전트에게 단순한 텍스트 생성 이상의 구체적인 능력을 부여하는 기본 구성 요소입니다. 일반적으로 특정 동작(예: API 호출, 데이터베이스 조회, 계산 수행 등)을 수행하는 일반적인 Python 함수입니다.

우리의 첫 번째 도구은 *mock* 날씨 보고서를 제공할 것입니다. 이렇게 하면 외부 API 키 없이 에이전트 구조에 집중할 수 있습니다. 이후에는 이 mock 함수를 실제 날씨 서비스를 호출하는 함수로 쉽게 교체할 수 있습니다.

**핵심 개념: Docstring이 매우 중요합니다\!** 에이전트의 LLM은 함수의 **docstring**을 기반으로 다음을 이해합니다:

* *무엇을* 하는 도구인지  
* *언제* 사용해야 하는지  
* 어떤 인자(`city: str`)가 필요한지  
* 어떤 정보를 반환하는지

**Best Practice:** 도구에 대해 명확하고 설명적이며 정확한 docstring을 작성하세요. LLM이 도구를 올바르게 사용하기 위해 필수적입니다.

In [ ]:
# @title Define the get_weather Tool
def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city (e.g., "New York", "London", "Tokyo").

    Returns:
        dict: A dictionary containing the weather information.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'report' key with weather details.
              If 'error', includes an 'error_message' key.
    """
    print(f"--- Tool: get_weather called for city: {city} ---") # Log tool execution
    city_normalized = city.lower().replace(" ", "") # Basic normalization

    # Mock weather data
    mock_weather_db = {
        "newyork": {"status": "success", "report": "The weather in New York is sunny with a temperature of 25°C."},
        "london": {"status": "success", "report": "It's cloudy in London with a temperature of 15°C."},
        "tokyo": {"status": "success", "report": "Tokyo is experiencing light rain and a temperature of 18°C."},
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"Sorry, I don't have weather information for '{city}'."}

# Example tool usage (optional test)
print(get_weather("New York"))
print(get_weather("Paris"))

---

**2\. 에이전트 정의하기 (`weather_agent`)**

이제 **에이전트** 자체를 만들어봅시다. ADK에서 `Agent`는 사용자, LLM, 그리고 사용 가능한 도구들 간의 상호작용을 조율합니다.

에이전트는 여러 핵심 파라미터로 구성됩니다:

* `name`: 이 에이전트를 식별하는 고유한 이름 (예: "weather\_agent\_v1")  
* `model`: 사용할 LLM 지정 (예: `MODEL_GEMINI_2_5_PRO`). 우리는 Gemini 모델로 시작합니다.  
* `description`: 에이전트의 전체 목적에 대한 간결한 요약. 이 설명은 나중에 다른 에이전트들이 *이* 에이전트에게 작업을 위임할지를 결정할 때 매우 중요합니다.  
* `instruction`: LLM이 어떤 방식으로 동작해야 하는지, 어떤 페르소나를 가졌는지, 목표는 무엇인지, 특히 *어떻게 그리고 언제* 할당된 `tools`를 사용할지를 자세히 안내합니다.  
* `tools`: 에이전트가 사용할 수 있도록 허용된 실제 Python 도구 함수들의 리스트 (예: `[get_weather]`)

**Best Practice:** 명확하고 구체적인 `instruction` 프롬프트를 제공하세요. 지침이 자세할수록 LLM은 자신의 역할과 도구 사용 방법을 더 잘 이해할 수 있습니다. 필요하다면 오류 처리 방식도 명시적으로 작성하세요.

**Best Practice:** 설명적인 `name`과 `description` 값을 선택하세요. 이는 ADK 내부적으로 사용되며, 이후 자동 위임 기능(뒤에서 다룸)에 매우 중요합니다.

In [ ]:
# @title Weather Agent 정의하기
# 앞서서 정의한 모델 상수 중 하나를 사용하세요.
AGENT_MODEL = MODEL_GEMINI_2_0_FLASH # Gemini로 시작

weather_agent = Agent(
    name="weather_agent_v1",
    model=AGENT_MODEL, # Gemini인 경우 문자열, LiteLLM인 경우 객체
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly.",
    tools=[get_weather], # 함수를 직접 전달
)

print(f"Agent '{weather_agent.name}' created using model '{AGENT_MODEL}'.")

---

**3\. Runner 및 Session Service 설정하기**

에이전트를 실행하고 대화를 관리하려면 두 가지 구성 요소가 더 필요합니다:

* `SessionService`: 서로 다른 사용자 및 세션에 대한 대화 기록과 상태를 관리하는 역할을 합니다. `InMemorySessionService`는 모든 정보를 메모리에 저장하는 간단한 구현체로, 테스트나 간단한 애플리케이션에 적합합니다. 주고받은 메시지를 추적합니다. 상태 영속성에 대해서는 Step 4에서 더 자세히 다룹니다.  
* `Runner`: 상호작용 흐름을 조율하는 엔진입니다. 사용자 입력을 받아 적절한 에이전트에 전달하고, 에이전트의 로직에 따라 LLM 및 도구 호출을 관리하며, `SessionService`를 통해 세션 상태를 업데이트하고, 상호작용의 진행 상황을 나타내는 이벤트들을 생성합니다.

In [ ]:
# @title Session Service와 Runner 설정

# --- Session 관리 ---
# 핵심 컨셉: SessionService는 대화 히스토리와 상태를 저장합니다.
# 이 튜토리얼에서 사용하는 InMemorySessionService는 심플하고, 반영구적인 저장소입니다.
session_service = InMemorySessionService()

# 상호작용 컨텍스트를 식별하기 위한 상수를 정의합니다.
APP_NAME = "weather_tutorial_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # 단순함을 위해 고정된 세션ID 사용

# 대화가 발생할 특정한 Session 생성
session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
# 핵심 컨셉: Runner는 에이전트 실행 루프를 조율합니다.
runner = Runner(
    agent=weather_agent, # 우리가 실행하려는 에이전트
    app_name=APP_NAME,   # 실행을 우리 앱과 연관시킵니다.
    session_service=session_service # 우리의 Session 관리자를 사용합니다.
)
print(f"Runner created for agent '{runner.agent.name}'.")

---

**4\. 에이전트와 상호작용하기**

에이전트에게 메시지를 보내고 응답을 받기 위한 방법이 필요합니다. LLM 호출 및 도구 실행에는 시간이 걸릴 수 있기 때문에, ADK의 `Runner`는 비동기적으로 동작합니다.

우리는 `async` 헬퍼 함수(`call_agent_async`)를 정의할 것입니다. 이 함수는 다음을 수행합니다:

1. 사용자 질문 문자열을 입력으로 받습니다.  
2. 이를 ADK의 `Content` 형식으로 포장합니다.  
3. 사용자/세션 컨텍스트와 새 메시지를 전달하여 `runner.run_async`를 호출합니다.  
4. Runner가 생성하는 **Event**들을 반복(iterate)합니다. Event는 에이전트 실행의 각 단계를 나타냅니다 (예: 도구 호출 요청, 도구 결과 수신, 중간 LLM 사고, 최종 응답 등).  
5. `event.is_final_response()`를 사용하여 **최종 응답** 이벤트를 식별하고 출력합니다.

**왜 `async`인가?** LLM과 (외부 API와 같은) 도구와의 상호작용은 I/O 바운드 작업입니다. `asyncio`를 사용하면 이러한 작업을 효율적으로 처리할 수 있으며, 실행이 블로킹되지 않도록 할 수 있습니다.

In [ ]:
# @title 에이전트 상호작용 함수 정의하기

from google.genai import types # 메시지 Content/Parts를 생성하기 위함

async def call_agent_async(query: str, runner, user_id, session_id):
  """에이전트에 쿼리를 보내고, 최종 응답을 출력합니다."""
  print(f"\n>>> User Query: {query}")

  # 사용자 메시지를 ADK 형식으로 준비합니다.
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # 핵심 컨셉: run_async는 에이전트의 로직을 실행하고 Event들을 생성합니다.
  # 최종 답변을 찾기 위해 이벤트들을 반복(iterate)합니다.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # 아래 줄의 주석을 해제하면 실행 중 발생하는 *모든* 이벤트를 확인할 수 있습니다.
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # 핵심 컨셉: is_final_response()는 해당 턴의 마지막 메시지임을 나타냅니다.
      if event.is_final_response():
          if event.content and event.content.parts:
             # 처음 부분에 텍스트 응답이 있다고 가정합니다.
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # 잠재적인 오류나 에스컬레이션 상황을 처리합니다.
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # 필요하다면 여기에서 추가적인 확인을 수행하세요 (예: 특정 오류 코드 등).
          break # 최종 응답을 찾으면 이벤트 처리를 중단합니다.

  print(f"<<< Agent Response: {final_response_text}")

---

**5\. 대화 실행하기**

마지막으로, 에이전트에게 몇 가지 질문을 보내며 설정을 테스트해봅시다. `async` 호출들을 하나의 메인 `async` 함수로 감싸고, 이를 `await`를 사용해 실행합니다.

출력을 확인해보세요:

* 사용자 질문을 확인할 수 있습니다.  
* 에이전트가 도구를 사용할 때 `--- Tool: get_weather called... ---` 로그가 출력됩니다.  
* 에이전트의 최종 응답을 확인하세요. 특히 파리(Paris)에 대한 날씨 데이터가 없을 때 어떻게 처리하는지도 주목해보세요.

In [ ]:
# @title 첫 대화 실행

# 상호작용 헬퍼를 기다리기 위해 async 함수가 필요합니다.
async def run_conversation():
    await call_agent_async("What is the weather like in London?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

    await call_agent_async("How about Paris?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID) # 도구의 오류 메시지를 기대

    await call_agent_async("Tell me the weather in New York",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

# Colab이나 Jupyter와 같은 async 컨텍스트에서 `await`를 사용하여 대화를 실행합니다.
await run_conversation()

---

축하합니다! 여러분은 첫 번째 ADK 에이전트를 성공적으로 구축하고 상호작용까지 마쳤습니다. 이 에이전트는 사용자의 요청을 이해하고, 정보를 찾기 위해 도구를 사용하며, 도구의 결과에 따라 적절히 응답합니다.

다음 단계에서는 이 에이전트를 구동하는 언어 모델(Language Model)을 쉽게 교체하는 방법을 살펴보겠습니다.

## Step 2: LiteLLM을 이용한 멀티 모델

Step 1에서는 특정 Gemini 모델을 기반으로 동작하는 기능성 Weather Agent를 구축했습니다. 효과적이긴 하지만, 실제 환경에서는 *다양한* 대규모 언어 모델(LLM)을 유연하게 사용할 수 있는 능력이 더욱 유리합니다. 그 이유는 다음과 같습니다:

*   **성능:** 일부 모델은 특정 작업(예: 코딩, 추론, 창의적 글쓰기)에 뛰어난 성능을 발휘합니다.
*   **비용:** 모델마다 가격이 다릅니다.
*   **기능:** 각 모델은 서로 다른 기능, 컨텍스트 윈도우 크기, 파인튜닝 옵션 등을 제공합니다.
*   **가용성/이중화:** 대안 모델을 보유하면 한 모델 제공자에 문제가 생기더라도 애플리케이션이 계속 작동할 수 있습니다.

ADK는 [**LiteLLM**](https://github.com/BerriAI/litellm) 라이브러리와의 통합을 통해 다양한 모델 간 전환을 매우 간편하게 만들어줍니다. LiteLLM은 100개 이상의 LLM을 하나의 일관된 인터페이스로 제공합니다.

**이번 단계에서는 다음을 수행합니다:**

1.  `LiteLlm` 래퍼를 사용하여 OpenAI(GPT), Anthropic(Claude) 등 다양한 제공자의 모델을 ADK `Agent`에 설정하는 방법을 학습합니다.  
2.  각각 고유한 세션과 러너(runner)를 갖는 Weather Agent 인스턴스를 정의하고 구성하며, 서로 다른 LLM을 기반으로 즉시 테스트합니다.  
3.  이러한 다양한 에이전트들과 상호작용하여, 동일한 도구를 사용하더라도 응답이 어떻게 달라질 수 있는지 관찰합니다.

---

**1\. `LiteLlm` 불러오기**

이것은 초기 설정(Step 0)에서 이미 임포트했지만, 멀티 모델 지원을 위한 핵심 구성 요소입니다:

In [ ]:
# @title 1. LiteLlm 불러오기
from google.adk.models.lite_llm import LiteLlm

**2\. 멀티 모델 에이전트 정의 및 테스트**

이제 단순히 모델 이름 문자열만 전달하는 대신(기본값은 구글의 Gemini 모델), 원하는 모델 식별자 문자열을 `LiteLlm` 클래스 내에 감싸서 전달합니다.

*   **핵심 개념: `LiteLlm` 래퍼:** `LiteLlm(model="provider/model_name")` 문법은 ADK에게 해당 에이전트의 요청을 LiteLLM 라이브러리를 통해 지정된 모델 제공자로 라우팅하라고 지시합니다.

OpenAI와 Anthropic의 API 키가 Step 0에서 설정되었는지 확인하세요. 우리는 앞서 정의한 `call_agent_async` 함수를 사용하여(이제 `runner`, `user_id`, `session_id`를 인자로 받음), 각각의 에이전트 설정 직후에 즉시 상호작용을 수행합니다.

아래 각 블록은 다음을 수행합니다:

*   특정 LiteLLM 모델(`MODEL_GPT_4O` 또는 `MODEL_CLAUDE_SONNET`)을 사용하여 에이전트를 정의합니다.  
*   해당 에이전트의 테스트 실행을 위한 *새롭고 분리된* `InMemorySessionService`와 세션을 생성합니다. 이를 통해 각 대화 기록이 독립적으로 유지됩니다.  
*   해당 에이전트와 세션 서비스를 기반으로 한 `Runner`를 생성합니다.  
*   `call_agent_async`를 즉시 호출하여 쿼리를 전송하고 에이전트를 테스트합니다.

**Best Practice:** 모델 이름은 `MODEL_GPT_4O`, `MODEL_CLAUDE_SONNET`처럼 상수로 정의해 사용하는 것이 좋습니다. 이는 오타를 방지하고 코드 관리를 쉽게 해줍니다.

**오류 처리:** 에이전트 정의는 `try...except` 블록으로 감싸줍니다. 이를 통해 특정 제공자의 API 키가 누락되었거나 유효하지 않은 경우 전체 코드 실행이 실패하지 않고, 설정된 모델만으로도 튜토리얼을 계속 진행할 수 있습니다.

먼저, OpenAI의 GPT-4o를 사용한 에이전트를 생성하고 테스트해봅시다.

In [ ]:
# @title 테스트 GPT Agent 정의

# Step 1에서 정의한 `get_weather` 함수가 현재 환경에 정의되어 있는지 확인하세요.
# 앞서 정의한 `call_agent_async` 함수도 준비되어 있어야 합니다.

# --- Agent using GPT-4o ---
weather_agent_gpt = None # None으로 초기화
runner_gpt = None      # Runner를 None으로 초기화

try:
    weather_agent_gpt = Agent(
        name="weather_agent_gpt",
        # 핵심 차이점: 모델 식별자를 LiteLlm으로 랩핑
        model=LiteLlm(model=MODEL_GPT_4O),
        description="Provides weather information (using GPT-4o).",
        instruction="You are a helpful weather assistant powered by GPT-4o. "
                    "Use the 'get_weather' tool for city weather requests. "
                    "Clearly present successful reports or polite error messages based on the tool's output status.",
        tools=[get_weather], # 동일한 도구 재사용
    )
    print(f"Agent '{weather_agent_gpt.name}' created using model '{MODEL_GPT_4O}'.")

    # 이 튜토리얼에서 사용하는 InMemorySessionService는 심플하고, 반영구적인 저장소입니다.
    session_service_gpt = InMemorySessionService() # 전용 서비스를 생성합니다.

    # 상호작용 컨텍스트를 식별하기 위한 상수를 정의합니다.
    APP_NAME_GPT = "weather_tutorial_app_gpt" # 이번 테스트를 위한 고유한 앱이름
    USER_ID_GPT = "user_1_gpt"
    SESSION_ID_GPT = "session_001_gpt" # 단순함을 위해 고정된 세션ID 사용

    # 대화가 발생할 특정한 Session 생성
    session_gpt = session_service_gpt.create_session(
        app_name=APP_NAME_GPT,
        user_id=USER_ID_GPT,
        session_id=SESSION_ID_GPT
    )
    print(f"Session created: App='{APP_NAME_GPT}', User='{USER_ID_GPT}', Session='{SESSION_ID_GPT}'")

    # 이 에이전트와 해당 세션 서비스를 위한 전용 runner를 생성합니다.
    runner_gpt = Runner(
        agent=weather_agent_gpt,
        app_name=APP_NAME_GPT,       # 특정한 앱이름 사용
        session_service=session_service_gpt # 특정한 Session Service 사용
        )
    print(f"Runner created for agent '{runner_gpt.agent.name}'.")

    # --- GPT Agent 테스트 ---
    print("\n--- Testing GPT Agent ---")
    # call_agent_async 함수가 올바른 runner, user_id, session_id를 사용하도록 확인하세요.
    await call_agent_async(query = "What's the weather in Tokyo?",
                           runner=runner_gpt,
                           user_id=USER_ID_GPT,
                           session_id=SESSION_ID_GPT)

except Exception as e:
    print(f"❌ Could not create or run GPT agent '{MODEL_GPT_4O}'. Check API Key and model name. Error: {e}")


다음으로, Anthropic의 Claude Sonnet에 대해서도 동일한 작업을 수행하겠습니다.

In [ ]:
# @title Claude 정의 및 테스트

# Step 1에서 정의한 `get_weather` 함수가 현재 환경에 정의되어 있는지 확인하세요.
# 앞서 정의한 `call_agent_async` 함수도 준비되어 있어야 합니다.

# --- Agent using Claude Sonnet ---
weather_agent_claude = None # None으로 초기화
runner_claude = None     # Runner를 None으로 초기화

try:
    weather_agent_claude = Agent(
        name="weather_agent_claude",
        # 핵심 차이점: 모델 식별자를 LiteLlm으로 랩핑
        model=LiteLlm(model=MODEL_CLAUDE_SONNET),
        description="Provides weather information (using Claude Sonnet).",
        instruction="You are a helpful weather assistant powered by Claude Sonnet. "
                    "Use the 'get_weather' tool for city weather requests. "
                    "Analyze the tool's dictionary output ('status', 'report'/'error_message'). "
                    "Clearly present successful reports or polite error messages.",
        tools=[get_weather], # 동일한 도구 재사용
    )
    print(f"Agent '{weather_agent_claude.name}' created using model '{MODEL_CLAUDE_SONNET}'.")

    # 이 튜토리얼에서 사용하는 InMemorySessionService는 심플하고, 반영구적인 저장소입니다.
    session_service_claude = InMemorySessionService() # 전용 서비스를 생성합니다.

    # 상호작용 컨텍스트를 식별하기 위한 상수를 정의합니다.
    APP_NAME_CLAUDE = "weather_tutorial_app_claude" # 고유한 앱 이름
    USER_ID_CLAUDE = "user_1_claude"
    SESSION_ID_CLAUDE = "session_001_claude" # 단순함을 위해 고정된 세션ID 사용

    # 대화가 발생할 특정한 Session 생성
    session_claude = session_service_claude.create_session(
        app_name=APP_NAME_CLAUDE,
        user_id=USER_ID_CLAUDE,
        session_id=SESSION_ID_CLAUDE
    )
    print(f"Session created: App='{APP_NAME_CLAUDE}', User='{USER_ID_CLAUDE}', Session='{SESSION_ID_CLAUDE}'")

    # 이 에이전트와 해당 세션 서비스를 위한 전용 runner를 생성합니다.
    runner_claude = Runner(
        agent=weather_agent_claude,
        app_name=APP_NAME_CLAUDE,       # 특정한 앱이름 사용
        session_service=session_service_claude # 특정한 Session Service 사용
        )
    print(f"Runner created for agent '{runner_claude.agent.name}'.")

    # --- Claude Agent 테스트 ---
    print("\n--- Testing Claude Agent ---")
    # call_agent_async 함수가 올바른 runner, user_id, session_id를 사용하도록 확인하세요.
    await call_agent_async(query = "Weather in London please.",
                           runner=runner_claude,
                           user_id=USER_ID_CLAUDE,
                           session_id=SESSION_ID_CLAUDE)

except Exception as e:
    print(f"❌ Could not create or run Claude agent '{MODEL_CLAUDE_SONNET}'. Check API Key and model name. Error: {e}")

출력 결과를 두 코드 블록에서 주의 깊게 확인해보세요. 다음과 같은 점들을 확인할 수 있어야 합니다:

1.  각 에이전트(`weather_agent_gpt`, `weather_agent_claude`)가 성공적으로 생성됩니다 (API 키가 유효한 경우).  
2.  각각에 대해 전용 세션과 runner가 설정됩니다.  
3.  각 에이전트는 쿼리를 처리할 때 `get_weather` 도구를 사용해야 함을 올바르게 인식합니다 (`--- Tool: get_weather called... ---` 로그가 출력됩니다).  
4.  *기저 도구 로직(tool logic)*은 동일하게 유지되며, 항상 mock 데이터를 반환합니다.  
5.  하지만, 각 에이전트가 생성하는 **최종 텍스트 응답**은 표현 방식, 어조, 형식 등에서 약간 다를 수 있습니다. 이는 동일한 instruction prompt를 서로 다른 LLM(GPT-4o vs. Claude Sonnet)이 해석하고 실행하기 때문입니다.

이번 단계는 ADK + LiteLLM이 제공하는 강력함과 유연성을 보여줍니다. 핵심 애플리케이션 로직(도구, 에이전트 구조)은 그대로 유지하면서도 다양한 LLM을 실험하고 배포하는 것이 매우 간편합니다.

다음 단계에서는 단일 에이전트를 넘어서, 에이전트들이 서로에게 작업을 위임할 수 있는 작은 팀을 구축해보겠습니다!

---

## Step 3: 에이전트 팀 빌딩 \- 인사 & 작별인사에게 위임하기

Step 1과 Step 2에서는 오직 날씨 조회에만 집중하는 단일 에이전트를 만들고 실험해보았습니다. 특정 작업에는 효과적이지만, 실제 애플리케이션에서는 훨씬 다양한 사용자 상호작용을 처리해야 하는 경우가 많습니다. 하나의 에이전트에 도구와 복잡한 지침을 계속 추가할 수도 있지만, 이는 곧 관리가 어렵고 비효율적으로 변할 수 있습니다.

보다 견고한 접근 방식은 **Agent Team**을 구축하는 것입니다. 이는 다음과 같은 방식으로 구성됩니다:

1. 각기 다른 기능(예: 날씨, 인사, 계산 등)을 담당하는 **전문화된 에이전트**를 여러 개 생성합니다.  
2. 사용자로부터 처음 요청을 받는 **루트 에이전트(root agent)** 또는 조정자(agent orchestrator)를 지정합니다.  
3. 루트 에이전트가 사용자 의도에 따라 요청을 가장 적절한 하위 에이전트에게 **위임(delegate)** 할 수 있도록 합니다.

**왜 Agent Team을 구축하나요?**

* **모듈성(Modularity):** 개별 에이전트를 더 쉽게 개발, 테스트, 유지보수할 수 있습니다.  
* **전문화(Specialization):** 각 에이전트는 해당 작업에 맞는 지침과 모델로 최적화될 수 있습니다.  
* **확장성(Scalability):** 새로운 기능을 추가할 때 새로운 에이전트만 추가하면 됩니다.  
* **효율성(Efficiency):** 인사와 같은 단순 작업에는 더 간단하거나 저렴한 모델을 사용할 수 있습니다.

**이번 단계에서는 다음을 수행합니다:**

1. 인사(`say_hello`)와 작별 인사(`say_goodbye`)를 처리하는 간단한 도구(tool)를 정의합니다.  
2. 두 개의 새로운 전문화 하위 에이전트 `greeting_agent`와 `farewell_agent`를 생성합니다.  
3. 기존의 날씨 에이전트를 `weather_agent_v2`로 업데이트하고, **루트 에이전트** 역할을 하도록 만듭니다.  
4. 루트 에이전트에 하위 에이전트들을 구성하고, **자동 위임(automatic delegation)** 기능을 활성화합니다.  
5. 루트 에이전트에게 다양한 유형의 요청을 보내어 위임 흐름이 잘 작동하는지 테스트합니다.

---

**1\. 하위 에이전트를 위한 도구 정의하기**

먼저, 새로운 전문화된 에이전트들이 사용할 간단한 Python 함수들을 생성해봅시다. 이 함수들은 각각 도구(tools)로 작용하게 됩니다.  
잊지 마세요 — 명확한 docstring은 해당 도구를 사용할 에이전트에게 매우 중요합니다.

In [ ]:
# @title 인사와 작별인사 에이전트를 위한 도구 정의

# 이 단계를 독립적으로 실행하는 경우, Step 1에서 정의한 'get_weather' 함수가 사용 가능한지 확인하세요.
# def get_weather(city: str) -> dict: ... (from Step 1)

def say_hello(name: str = "there") -> str:
    """Provides a simple greeting, optionally addressing the user by name.

    Args:
        name (str, optional): The name of the person to greet. Defaults to "there".

    Returns:
        str: A friendly greeting message.
    """
    print(f"--- Tool: say_hello called with name: {name} ---")
    return f"Hello, {name}!"

def say_goodbye() -> str:
    """Provides a simple farewell message to conclude the conversation."""
    print(f"--- Tool: say_goodbye called ---")
    return "Goodbye! Have a great day."

print("Greeting and Farewell tools defined.")

# 선택적인 자가 점검
print(say_hello("Alice"))
print(say_goodbye())

---

**2\. 하위 에이전트 정의하기 (Greeting & Farewell)**

이제 각 전문 작업을 담당할 `Agent` 인스턴스를 생성합니다. 이들의 `instruction`은 매우 집중되어 있으며, 무엇보다 `description`이 명확하게 작성되어야 합니다.  
`description`은 *루트 에이전트*가 해당 요청을 언제 이 하위 에이전트에게 위임할지를 결정할 때 사용하는 주요 기준입니다.

하위 에이전트에는 서로 다른 LLM을 사용할 수도 있습니다! 여기서는 인사(Greeting) 에이전트와 작별(Farewell) 에이전트 모두에 GPT-4o를 사용하겠습니다.  
(원한다면 Claude나 Gemini로 바꾸는 것도 간단하며, API 키가 설정되어 있다면 바로 가능합니다.)

**Best Practice:** 하위 에이전트의 `description` 필드는 해당 에이전트의 기능을 정확하고 간결하게 요약해야 합니다. 이는 자동 위임의 효과성을 결정짓는 핵심 요소입니다.

**Best Practice:** 하위 에이전트의 `instruction` 필드는 제한된 역할 범위에 맞춰 작성되어야 하며, 무엇을 *해야 하는지*뿐만 아니라 *하면 안 되는 것*도 명확히 지시해야 합니다 (예: "당신의 *유일한* 임무는...").

In [ ]:
# @title 인사와 작별 하위 에이전트 정의

# LiteLlm이 임포트 되어 있고, API 키가 설정되어 있음을 확인하세요. (from Step 0/2)
# from google.adk.models.lite_llm import LiteLlm
# MODEL_GPT_4O, MODEL_CLAUDE_SONNET 등이 설정되어 있어야 합니다.

# --- Greeting Agent ---
greeting_agent = None
try:
    greeting_agent = Agent(
        # 단순한 작업에는 더 간단하거나 저렴한 모델을 사용하는 것도 가능합니다.
        model=LiteLlm(model=MODEL_GPT_4O),
        name="greeting_agent",
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting to the user. "
                    "Use the 'say_hello' tool to generate the greeting. "
                    "If the user provides their name, make sure to pass it to the tool. "
                    "Do not engage in any other conversation or tasks.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.", # 위임을 위해 매우 중요
        tools=[say_hello],
    )
    print(f"✅ Agent '{greeting_agent.name}' created using model '{MODEL_GPT_4O}'.")
except Exception as e:
    print(f"❌ Could not create Greeting agent. Check API Key ({MODEL_GPT_4O}). Error: {e}")

# --- Farewell Agent ---
farewell_agent = None
try:
    farewell_agent = Agent(
        # 같거나 다른 모델 사용 가능
        model=LiteLlm(model=MODEL_GPT_4O), # 이 예제에서는 GPT로 유지합니다.
        name="farewell_agent",
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message. "
                    "Use the 'say_goodbye' tool when the user indicates they are leaving or ending the conversation "
                    "(e.g., using words like 'bye', 'goodbye', 'thanks bye', 'see you'). "
                    "Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.", # 위임을 위해 매우 중요
        tools=[say_goodbye],
    )
    print(f"✅ Agent '{farewell_agent.name}' created using model '{MODEL_GPT_4O}'.")
except Exception as e:
    print(f"❌ Could not create Farewell agent. Check API Key ({MODEL_GPT_4O}). Error: {e}")

---

**3\. 하위 에이전트를 포함한 루트 에이전트 정의하기 (Weather Agent v2)**

이제 기존의 `weather_agent`를 업그레이드합니다. 주요 변경 사항은 다음과 같습니다:

* `sub_agents` 파라미터 추가: 방금 생성한 `greeting_agent`와 `farewell_agent` 인스턴스를 리스트로 전달합니다.  
* `instruction` 업데이트: 루트 에이전트에게 하위 에이전트들의 존재와, *언제* 이들에게 작업을 위임해야 하는지를 명시적으로 알려줍니다.

**핵심 개념: 자동 위임 (Auto Flow)**  
`sub_agents` 리스트를 제공함으로써, ADK는 자동 위임 기능을 활성화합니다. 루트 에이전트가 사용자 쿼리를 받으면, 해당 LLM은 자신의 지침과 도구뿐 아니라 각 하위 에이전트의 `description`도 함께 고려합니다.  
LLM이 어떤 쿼리가 특정 하위 에이전트의 기능 설명(예: “간단한 인사를 처리합니다”)에 더 적합하다고 판단되면, *그 턴에 대한 제어권을 해당 하위 에이전트로 이전*하는 특별한 내부 동작을 자동으로 생성합니다. 그러면 하위 에이전트는 자신의 모델, 지침, 도구를 사용해 쿼리를 처리합니다.

**Best Practice:** 루트 에이전트의 `instruction`에는 위임 기준이 명확히 드러나야 합니다. 하위 에이전트의 이름을 직접 언급하고, 어떤 조건에서 위임이 일어나야 하는지를 구체적으로 설명하세요.

In [ ]:
# @title 하위 에이전트를 포함한 루트 에이전트 정의하기

# 루트 에이전트를 정의하기 전에 하위 에이전트들이 정상적으로 생성되었는지 확인하세요.
# 또한, 원래의 `get_weather` 도구가 정의되어 있는지도 확인하세요.
root_agent = None
runner_root = None # runner 초기화

if greeting_agent and farewell_agent and 'get_weather' in globals():
    # 조율 역할을 담당할 루트 에이전트에는 성능이 뛰어난 Gemini 모델을 사용해봅시다.
    root_agent_model = MODEL_GEMINI_2_0_FLASH

    weather_agent_team = Agent(
        name="weather_agent_v2", # 새 버전의 이름 사용
        model=root_agent_model,
        description="The main coordinator agent. Handles weather requests and delegates greetings/farewells to specialists.",
        instruction="You are the main Weather Agent coordinating a team. Your primary responsibility is to provide weather information. "
                    "Use the 'get_weather' tool ONLY for specific weather requests (e.g., 'weather in London'). "
                    "You have specialized sub-agents: "
                    "1. 'greeting_agent': Handles simple greetings like 'Hi', 'Hello'. Delegate to it for these. "
                    "2. 'farewell_agent': Handles simple farewells like 'Bye', 'See you'. Delegate to it for these. "
                    "Analyze the user's query. If it's a greeting, delegate to 'greeting_agent'. If it's a farewell, delegate to 'farewell_agent'. "
                    "If it's a weather request, handle it yourself using 'get_weather'. "
                    "For anything else, respond appropriately or state you cannot handle it.",
        tools=[get_weather], # 루트 에이전트는 여전히 본연의 작업인 날씨 조회를 위해 get_weather 도구가 필요합니다.
        # 핵심 차이점: 여기서 하위 에이전트를 연결합니다!
        sub_agents=[greeting_agent, farewell_agent]
    )
    print(f"✅ Root Agent '{weather_agent_team.name}' created using model '{root_agent_model}' with sub-agents: {[sa.name for sa in weather_agent_team.sub_agents]}")

else:
    print("❌ Cannot create root agent because one or more sub-agents failed to initialize or 'get_weather' tool is missing.")
    if not greeting_agent: print(" - Greeting Agent is missing.")
    if not farewell_agent: print(" - Farewell Agent is missing.")
    if 'get_weather' not in globals(): print(" - get_weather function is missing.")



---

**4\. 에이전트 팀과 상호작용하기**

이제 루트 에이전트(`weather_agent_team` — *참고: 이전 코드 블록에서 이 이름이 `root_agent` 등으로 정의되었을 수 있으니 일치하는지 확인하세요*)와 전문화된 하위 에이전트들을 정의했으니, 위임(delegation) 메커니즘을 테스트해봅시다.

다음 코드 블록에서는 다음을 수행합니다:

1.  `async` 함수 `run_team_conversation`을 정의합니다.  
2.  함수 내에서 이 테스트 실행을 위한 *새로운 전용* `InMemorySessionService`와 세션(`session_001_agent_team`)을 생성합니다. 이는 팀 간 상호작용 테스트를 위한 대화 기록을 분리해줍니다.  
3.  루트 에이전트 `weather_agent_team`과 전용 세션 서비스를 사용하도록 구성된 `Runner` (`runner_agent_team`)를 생성합니다.  
4.  업데이트된 `call_agent_async` 함수를 이용해 다양한 유형의 쿼리(인사, 날씨 요청, 작별 인사)를 `runner_agent_team`에 전달합니다. 이때 명시적으로 runner, user ID, session ID를 함께 전달합니다.  
5.  `run_team_conversation` 함수를 즉시 실행합니다.

예상되는 흐름:

1.  `"Hello there!"` 쿼리는 `runner_agent_team`에 전달됩니다.  
2.  루트 에이전트(`weather_agent_team`)는 이 요청을 받고, 자신의 지침과 `greeting_agent`의 `description`에 따라 작업을 위임합니다.  
3.  `greeting_agent`가 해당 쿼리를 처리하고, `say_hello` 도구를 호출해 응답을 생성합니다.  
4.  `"What is the weather in New York?"` 쿼리는 위임되지 않고, 루트 에이전트가 직접 `get_weather` 도구를 사용해 처리합니다.  
5.  `"Thanks, bye!"` 쿼리는 `farewell_agent`로 위임되어, `say_goodbye` 도구를 사용해 응답을 생성합니다.

In [ ]:
# @title 에이전트 팀과 상호작용하기

# 루트 에이전트('weather_agent_team' 또는 이전 셀에서 정의한 'root_agent' 등)가 정의되어 있는지 확인하세요.
# 또한, 'call_agent_async' 함수가 정의되어 있는지도 확인하세요.

# 루트 에이전트 변수가 존재하는지 확인한 후 대화 함수(conversation function)를 정의하세요.
root_agent_var_name = 'root_agent' # Step 3가이드의 기본 이름
if 'weather_agent_team' in globals(): # 이 이름이 대신 사용되었는지 확인
    root_agent_var_name = 'weather_agent_team'
elif 'root_agent' not in globals():
    print("⚠️ Root agent ('root_agent' or 'weather_agent_team') not found. Cannot define run_team_conversation.")
    # 코드 블록이 어쨌든 실행될 경우 NameError를 방지하기 위해 더미 값을 할당하세요.
    root_agent = None

if root_agent_var_name in globals() and globals()[root_agent_var_name]:
    async def run_team_conversation():
        print("\n--- Testing Agent Team Delegation ---")
        # InMemorySessionService는 이 튜토리얼에서 사용하는 간단한 비영속성 저장소입니다.
        session_service = InMemorySessionService()

        # 상호작용 컨텍스트를 식별하기 위한 상수를 정의합니다.
        APP_NAME = "weather_tutorial_agent_team"
        USER_ID = "user_1_agent_team"
        SESSION_ID = "session_001_agent_team" # 단순함을 위해 고정된 세션ID 사용

        # 대화가 진행될 특정 세션을 생성합니다.
        session = session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=SESSION_ID
        )
        print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

        # --- 실제 루트 에이전트 객체 가져오기 ---
        # 확인된 변수 이름을 사용하세요
        actual_root_agent = globals()[root_agent_var_name]

        # 이 에이전트 팀 테스트를 위한 전용 runner를 생성합니다
        runner_agent_team = Runner(
            agent=actual_root_agent, # 루트 에이전트 객체 사용
            app_name=APP_NAME,       # 특정한 앱 이름 사용
            session_service=session_service # 특정한 세션 서비스 사용
            )
        # 실제 루트 에이전트의 이름을 출력하도록 print 문을 수정했습니다
        print(f"Runner created for agent '{actual_root_agent.name}'.")

        # 항상 루트 에이전트의 runner를 통해 상호작용하며, 올바른 ID들을 전달하세요
        await call_agent_async(query = "Hello there!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "What is the weather in New York?",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "Thanks, bye!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)

    # 대화를 실행합니다.
    # 참고: 루트 및 하위 에이전트에서 사용하는 모델에 따라 API 키가 필요할 수 있습니다!
    await run_team_conversation()
else:
    print("\n⚠️ Skipping agent team conversation as the root agent was not successfully defined in the previous step.")


---

출력 로그를 자세히 확인해보세요. 특히 `--- Tool: ... called ---` 메시지에 주목하세요. 다음과 같은 동작을 관찰할 수 있어야 합니다:

*   `"Hello there!"` 요청에 대해 `say_hello` 도구가 호출됨 → `greeting_agent`가 이를 처리했음을 나타냅니다.  
*   `"What is the weather in New York?"` 요청에 대해 `get_weather` 도구가 호출됨 → 루트 에이전트가 직접 처리했음을 나타냅니다.  
*   `"Thanks, bye!"` 요청에 대해 `say_goodbye` 도구가 호출됨 → `farewell_agent`가 이를 처리했음을 나타냅니다.

이는 **자동 위임(automatic delegation)**이 성공적으로 이루어졌음을 보여줍니다!  
루트 에이전트는 자신의 지침과 하위 에이전트들의 `description`을 기반으로 사용자 요청을 적절한 전문화 에이전트에게 정확히 전달했습니다.

이제 여러분의 애플리케이션은 여러 에이전트가 협력하는 구조로 구성되었습니다. 이러한 모듈형 설계는 더 복잡하고 강력한 에이전트 시스템을 구축하는 데 있어 핵심적인 기반입니다.  
다음 단계에서는, 에이전트들이 대화 턴 사이에 정보를 기억할 수 있도록 **세션 상태(Session State)**를 부여해보겠습니다.

## Step 4: Session State를 이용한 기억과 개인화 추가하기

지금까지 에이전트 팀은 위임을 통해 다양한 작업을 처리할 수 있었지만, 각 상호작용은 항상 "새로 시작" 상태였습니다 — 에이전트는 과거 대화나 사용자 선호에 대한 기억이 없습니다. 보다 정교하고 문맥을 이해하는 경험을 만들기 위해서는 **기억(Memory)**이 필요합니다. ADK는 이를 위해 **Session State** 기능을 제공합니다.

**Session State란?**

* 이는 특정 사용자 세션(`APP_NAME`, `USER_ID`, `SESSION_ID`로 식별됨)에 연결된 Python 딕셔너리(`session.state`)입니다.  
* 하나의 세션 내에서 *여러 대화 턴에 걸쳐 정보를 지속*시킵니다.  
* 에이전트와 도구는 이 상태에 접근하여 읽거나 쓸 수 있으며, 이를 통해 세부 정보를 기억하고, 행동을 조정하며, 응답을 개인화할 수 있습니다.

**에이전트가 상태와 상호작용하는 방식:**

1. **`ToolContext` (주요 방법):** 도구는 `ToolContext` 객체를 인자로 받을 수 있으며 (마지막 인자로 선언되면 ADK가 자동으로 전달), 이를 통해 `tool_context.state`를 통해 세션 상태에 직접 접근할 수 있습니다. 도구 실행 중 선호 정보를 읽거나 결과를 저장할 수 있습니다.  
2. **`output_key` (자동 응답 저장):** 에이전트를 `output_key="your_key"`로 설정하면, ADK는 해당 턴의 최종 텍스트 응답을 자동으로 `session.state["your_key"]`에 저장합니다.

**이번 단계에서는 Weather Bot 팀을 다음과 같이 향상시킵니다:**

1. 상태를 독립적으로 실험할 수 있도록 **새로운** `InMemorySessionService`를 사용합니다.  
2. 세션 상태를 초기화하여 사용자 선호도(`temperature_unit`)를 지정합니다.  
3. 이 선호도를 `ToolContext`를 통해 읽고, 출력 형식을 섭씨/화씨로 조정하는 상태 인식 버전의 날씨 도구(`get_weather_stateful`)를 생성합니다.  
4. 루트 에이전트를 업데이트하여 이 상태 인식 도구를 사용하게 하고, 최종 날씨 응답을 자동으로 세션 상태에 저장하도록 `output_key`를 설정합니다.  
5. 대화를 실행하여 초기 상태가 도구에 어떤 영향을 미치는지, 상태 수동 변경이 이후 동작을 어떻게 바꾸는지, `output_key`가 어떻게 응답을 저장하는지를 확인합니다.

---

**1\. 새로운 세션 서비스 및 상태 초기화하기**

이전 단계의 영향 없이 상태 관리를 명확하게 시연하기 위해, 새로운 `InMemorySessionService` 인스턴스를 생성합니다.  
또한, 사용자의 선호 온도 단위를 정의한 초기 상태와 함께 세션을 생성합니다.

In [ ]:
# @title 1. 새로운 Session Service 및 State 초기화

# 세션 관련 구성요소 임포트
from google.adk.sessions import InMemorySessionService

# State 데모를 위한 새로운 InMemorySessionService 인스턴스 생성
session_service_stateful = InMemorySessionService()
print("✅ New InMemorySessionService created for state demonstration.")

# 튜토리얼 이 파트에서 사용할 고유 세션 ID 정의
SESSION_ID_STATEFUL = "session_state_demo_001"
USER_ID_STATEFUL = "user_state_demo"

# 초기 상태 데이터 정의 – 사용자 선호 온도 단위를 '섭씨(Celsius)'로 설정
initial_state = {
    "user_preference_temperature_unit": "Celsius"
}

# 세션 생성 및 초기 상태(state) 전달
session_stateful = session_service_stateful.create_session(
    app_name=APP_NAME, # 일관된 앱 이름 사용
    user_id=USER_ID_STATEFUL,
    session_id=SESSION_ID_STATEFUL,
    state=initial_state # <<< 세션 생성 시 초기 상태 설정
)
print(f"✅ Session '{SESSION_ID_STATEFUL}' created for user '{USER_ID_STATEFUL}'.")

# 세션 상태가 제대로 설정되었는지 확인
retrieved_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id=USER_ID_STATEFUL,
                                                         session_id = SESSION_ID_STATEFUL)
print("\n--- Initial Session State ---")
if retrieved_session:
    print(retrieved_session.state)
else:
    print("Error: Could not retrieve session.")

---

**2\. 상태 인식 날씨 도구 생성하기 (`get_weather_stateful`)**

이제 날씨 도구의 새로운 버전을 생성해봅니다. 이 버전의 핵심 기능은 `tool_context: ToolContext`를 인자로 받아 `tool_context.state`에 접근할 수 있다는 점입니다.  
이를 통해 `user_preference_temperature_unit` 값을 읽고, 이에 따라 온도를 섭씨 또는 화씨로 포맷합니다.

---

* **핵심 개념: `ToolContext`**  
  이 객체는 도구 로직과 세션 컨텍스트(상태 변수 읽기/쓰기 포함) 간의 다리를 제공합니다.  
  도구 함수의 마지막 인자로 정의하면 ADK가 자동으로 주입해줍니다.

* **Best Practice:**  
  상태에서 값을 읽을 때는 `dictionary.get('key', default_value)`를 사용하는 것이 좋습니다.  
  이렇게 하면 키가 아직 존재하지 않는 경우에도 도구가 오류 없이 안정적으로 동작할 수 있습니다.

In [ ]:
from google.adk.tools.tool_context import ToolContext

def get_weather_stateful(city: str, tool_context: ToolContext) -> dict:
    """Retrieves weather, converts temp unit based on session state."""
    print(f"--- Tool: get_weather_stateful called for {city} ---")

    # --- State로부터 선호도 확인 ---
    preferred_unit = tool_context.state.get("user_preference_temperature_unit", "Celsius") # 기본은 섭씨로 설정
    print(f"--- Tool: Reading state 'user_preference_temperature_unit': {preferred_unit} ---")

    city_normalized = city.lower().replace(" ", "")

    # Mock 날씨 데이터(모두 섭씨로 저장)
    mock_weather_db = {
        "newyork": {"temp_c": 25, "condition": "sunny"},
        "london": {"temp_c": 15, "condition": "cloudy"},
        "tokyo": {"temp_c": 18, "condition": "light rain"},
    }

    if city_normalized in mock_weather_db:
        data = mock_weather_db[city_normalized]
        temp_c = data["temp_c"]
        condition = data["condition"]

        # State 선호도에 따라 포맷팅 수행
        if preferred_unit == "Fahrenheit":
            temp_value = (temp_c * 9/5) + 32 # 화씨 계산
            temp_unit = "°F"
        else: # 기본은 섭씨
            temp_value = temp_c
            temp_unit = "°C"

        report = f"The weather in {city.capitalize()} is {condition} with a temperature of {temp_value:.0f}{temp_unit}."
        result = {"status": "success", "report": report}
        print(f"--- Tool: Generated report in {preferred_unit}. Result: {result} ---")

        # 상태에 값을 다시 저장하는 예시 (이 도구에서는 선택 사항)
        tool_context.state["last_city_checked_stateful"] = city
        print(f"--- Tool: Updated state 'last_city_checked_stateful': {city} ---")

        return result
    else:
        # 도시를 못찾는 경우 처리
        error_msg = f"Sorry, I don't have weather information for '{city}'."
        print(f"--- Tool: City '{city}' not found. ---")
        return {"status": "error", "error_message": error_msg}

print("✅ State-aware 'get_weather_stateful' tool defined.")


---

**3\. 하위 에이전트 재정의 및 루트 에이전트 업데이트**

이 단계가 독립적으로 구성되고 올바르게 실행될 수 있도록 하기 위해, 먼저 Step 3에서 정의했던 `greeting_agent`와 `farewell_agent`를 동일하게 다시 정의합니다.  
그 후, 새로운 루트 에이전트 `weather_agent_v4_stateful`을 정의합니다:

* 새로운 `get_weather_stateful` 도구를 사용합니다.  
* 위임을 위해 인사 및 작별 인사 하위 에이전트를 포함합니다.  
* **중요하게도**, `output_key="last_weather_report"`를 설정하여, 최종 날씨 응답을 세션 상태에 자동으로 저장하도록 합니다.

In [ ]:
# @title 3. 하위 에이전트 재정의 및 루트 에이전트 업데이트 (output_key 포함)

# 필요한 임포트 확인: Agent, LiteLlm, Runner
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.runners import Runner
# 도구 'say_hello', 'say_goodbye' 가 정의되었는지 확인 (from Step 3)
# MODEL_GPT_4O, MODEL_GEMINI_2_5_PRO 등 모델 상수가 정의되었는지 확인

# --- 인사 에이전트 재정의 (from Step 3) ---
greeting_agent = None
try:
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent",
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Agent '{greeting_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Greeting agent. Error: {e}")

# --- 작별 에이전트 재정의 (from Step 3) ---
farewell_agent = None
try:
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent",
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye' tool. Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Agent '{farewell_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Farewell agent. Error: {e}")

# --- 업데이트된 루트 에이전트 정의 ---
root_agent_stateful = None
runner_root_stateful = None # Runner 초기화

# 루트 에이전트를 생성하기 전에 필요한 요소들이 정의되어 있는지 확인
if greeting_agent and farewell_agent and 'get_weather_stateful' in globals():

    root_agent_model = MODEL_GEMINI_2_0_FLASH # 조율 모델 선택

    root_agent_stateful = Agent(
        name="weather_agent_v4_stateful", # 새로운 버전 이름
        model=root_agent_model,
        description="Main agent: Provides weather (state-aware unit), delegates greetings/farewells, saves report to state.",
        instruction="You are the main Weather Agent. Your job is to provide weather using 'get_weather_stateful'. "
                    "The tool will format the temperature based on user preference stored in state. "
                    "Delegate simple greetings to 'greeting_agent' and farewells to 'farewell_agent'. "
                    "Handle only weather requests, greetings, and farewells.",
        tools=[get_weather_stateful], # State-aware 도구 사용
        sub_agents=[greeting_agent, farewell_agent], # 하위 에이전트 포함
        output_key="last_weather_report" # <<< 에이전트의 최종 날씨 대답 자동 저장
    )
    print(f"✅ Root Agent '{root_agent_stateful.name}' created using stateful tool and output_key.")

    # --- 이 루트 에이전트와 새로운 세션 서비스를 위한 Runner 생성 ---
    runner_root_stateful = Runner(
        agent=root_agent_stateful,
        app_name=APP_NAME,
        session_service=session_service_stateful # 새로운 상태 기반(stateful) 세션 서비스 사용
    )
    print(f"✅ Runner created for stateful root agent '{runner_root_stateful.agent.name}' using stateful session service.")

else:
    print("❌ Cannot create stateful root agent. Prerequisites missing.")
    if not greeting_agent: print(" - greeting_agent definition missing.")
    if not farewell_agent: print(" - farewell_agent definition missing.")
    if 'get_weather_stateful' not in globals(): print(" - get_weather_stateful tool missing.")


---

**4\. 상태 흐름 테스트 및 상호작용하기**

이제 상태 기반 에이전트(`runner_root_stateful`)와 `session_service_stateful`을 사용해 **상태 상호작용 흐름**을 테스트하는 대화를 실행해봅니다.  
앞서 정의한 `call_agent_async` 함수를 사용하며, 반드시 올바른 `runner`, 사용자 ID(`USER_ID_STATEFUL`), 세션 ID(`SESSION_ID_STATEFUL`)를 전달해야 합니다.

### ✅ 예상 대화 흐름:

1. **날씨 확인 (London):**  
   `get_weather_stateful` 도구는 Section 1에서 초기화된 `"Celsius"` 상태 값을 읽어 섭씨로 날씨 정보를 반환해야 합니다.  
   이 응답은 루트 에이전트의 `output_key="last_weather_report"` 설정에 따라 `session.state['last_weather_report']`에 저장됩니다.

2. **상태 수동 업데이트:**  
   `session_service_stateful`의 내부 상태를 직접 수정하여 `"user_preference_temperature_unit"` 값을 `"Fahrenheit"`로 변경합니다.  
   🔍 **왜 직접 수정하나요?**  
   `get_session()`은 세션의 *복사본*을 반환하므로, 해당 객체를 수정해도 실제 세션 상태에 반영되지 않습니다.  
   `InMemorySessionService`를 사용할 때만 내부 `sessions` 딕셔너리에 직접 접근하는 것이 가능하며, 실제 저장된 상태를 수정할 수 있습니다.  
   ⚠️ 참고: 실제 서비스에서는 도구나 에이전트가 `EventActions(state_delta=...)`를 반환함으로써 상태를 변경합니다.

3. **다시 날씨 확인 (New York):**  
   이제 `get_weather_stateful` 도구는 수정된 `"Fahrenheit"` 상태를 읽어 화씨 온도로 변환하여 응답을 생성합니다.  
   이 응답은 `output_key`에 의해 `session.state['last_weather_report']`에 저장되며, 기존 섭씨 응답은 덮어씌워집니다.

4. **에이전트에게 인사:**  
   `greeting_agent`로의 위임이 여전히 잘 작동하는지 확인합니다.  
   이 인사 응답은 `output_key`에 의해 현재 세션의 마지막 응답으로 기록됩니다.

5. **최종 상태 점검:**  
   대화가 끝난 후, 세션을 한 번 더 가져와(`get_session()`으로 복사본 반환), 세션 상태를 출력합니다.  
   다음 항목을 확인해야 합니다:
   * `"user_preference_temperature_unit"` 값이 `"Fahrenheit"`로 설정되어 있음  
   * `"last_weather_report"` 값은 마지막 인사 응답  
   * `"last_city_checked_stateful"` 값은 도구가 마지막으로 확인한 도시

---

In [ ]:
# @title 4. 상태 흐름 및 output_key 테스트를 위한 상호작용

# 이전 셀에서 상태 유지 실행기(runner_root_stateful)가 사용 가능한지 확인하세요
# call_agent_async, USER_ID_STATEFUL, SESSION_ID_STATEFUL, APP_NAME이 정의되어 있는지 확인하세요

if 'runner_root_stateful' in globals() and runner_root_stateful:
  async def run_stateful_conversation():
      print("\n--- Testing State: Temp Unit Conversion & output_key ---")

      # 1. 날씨 확인 (초기 state 사용: Celsius)
      print("--- Turn 1: Requesting weather in London (expect Celsius) ---")
      await call_agent_async(query= "What's the weather in London?",
                             runner=runner_root_stateful,
                             user_id=USER_ID_STATEFUL,
                             session_id=SESSION_ID_STATEFUL
                            )

      # 2. State를 수동으로 업데이트하여 화씨를 선호하도록 변경 - 저장된 내용을 직접 변경
      print("\n--- Manually Updating State: Setting unit to Fahrenheit ---")
      try:
          # 내부 저장소에 직접 접근하세요 - **이는 테스트를 위한 InMemorySessionService에 한정됩니다**
          stored_session = session_service_stateful.sessions[APP_NAME][USER_ID_STATEFUL][SESSION_ID_STATEFUL]
          stored_session.state["user_preference_temperature_unit"] = "Fahrenheit"
          # 선택 사항: 어떤 로직이 타임스탬프에 의존한다면 타임스탬프도 업데이트하는 것이 좋습니다
          # import time
          # stored_session.last_update_time = time.time()
          print(f"--- Stored session state updated. Current 'user_preference_temperature_unit': {stored_session.state['user_preference_temperature_unit']} ---")
      except KeyError:
          print(f"--- Error: Could not retrieve session '{SESSION_ID_STATEFUL}' from internal storage for user '{USER_ID_STATEFUL}' in app '{APP_NAME}' to update state. Check IDs and if session was created. ---")
      except Exception as e:
           print(f"--- Error updating internal session state: {e} ---")

      # 3. 날씨를 다시 확인합니다 (도구는 이제 화씨를 사용해야 합니다)
      # 또한 output_key를 통해 'last_weather_report'가 업데이트됩니다
      print("\n--- Turn 2: Requesting weather in New York (expect Fahrenheit) ---")
      await call_agent_async(query= "Tell me the weather in New York.",
                             runner=runner_root_stateful,
                             user_id=USER_ID_STATEFUL,
                             session_id=SESSION_ID_STATEFUL
                            )

      # 4. 기본 위임을 테스트합니다 (여전히 작동해야 합니다)
      # 이는 'last_weather_report'를 다시 업데이트하며, 뉴욕 날씨 보고서를 덮어씁니다
      print("\n--- Turn 3: Sending a greeting ---")
      await call_agent_async(query= "Hi!",
                             runner=runner_root_stateful,
                             user_id=USER_ID_STATEFUL,
                             session_id=SESSION_ID_STATEFUL
                            )

  # 대화 실행
  await run_stateful_conversation()

  # 대화를 마친 후 최종 State 확인
  print("\n--- Inspecting Final Session State ---")
  final_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                       user_id= USER_ID_STATEFUL,
                                                       session_id=SESSION_ID_STATEFUL)
  if final_session:
      print(f"Final Preference: {final_session.state.get('user_preference_temperature_unit')}")
      print(f"Final Last Weather Report (from output_key): {final_session.state.get('last_weather_report')}")
      print(f"Final Last City Checked (by tool): {final_session.state.get('last_city_checked_stateful')}")
      # 전체 State를 출력해 자세한 내용 확인
      # print(f"Full State: {final_session.state}")
  else:
      print("\n❌ Error: Could not retrieve final session state.")

else:
  print("\n⚠️ Skipping state test conversation. Stateful root agent runner ('runner_root_stateful') is not available.")

---

대화 흐름과 마지막 세션 상태 출력 결과를 검토함으로써 다음을 확인할 수 있습니다:

*   **상태 읽기:** 날씨 도구(`get_weather_stateful`)는 상태에서 `user_preference_temperature_unit`을 올바르게 읽어, 처음 런던에 대해 "섭씨"를 사용했습니다.  
*   **상태 업데이트:** 직접 수정으로 저장된 선호도가 "화씨"로 성공적으로 변경되었습니다.  
*   **업데이트된 상태 읽기:** 이후 도구는 뉴욕의 날씨를 요청받았을 때 "화씨"를 읽어 변환을 수행했습니다.  
*   **도구 상태 쓰기:** 도구는 `tool_context.state`를 통해 `last_city_checked_stateful` ("New York")을 상태에 성공적으로 저장했습니다.  
*   **위임:** 상태가 수정된 이후에도 "Hi!" 요청에 대해 `greeting_agent`로의 위임이 올바르게 작동했습니다.  
*   **`output_key`:** `output_key="last_weather_report"`는 루트 에이전트가 최종적으로 응답한 각 턴의 응답을 상태에 성공적으로 저장했습니다. 이 흐름에서 마지막 응답은 인사("Hello, there!")였기 때문에 해당 응답이 상태 키를 덮어썼습니다.  
*   **최종 상태:** 마지막 확인에서는 선호도가 "화씨"로 유지되었음을 보여줍니다.  

이제 `ToolContext`를 사용해 에이전트의 행동을 개인화할 수 있도록 세션 상태를 통합하고, `InMemorySessionService`를 테스트하기 위해 상태를 수동으로 조작했으며, `output_key`를 통해 에이전트의 마지막 응답을 상태에 저장하는 간단한 메커니즘을 확인했습니다. 이러한 상태 관리의 기초는 다음 단계에서 콜백을 사용해 안전 가드를 구현하는 데 중요한 기반이 됩니다.

---

## Step 5: 안전 장치 추가 – `before_model_callback`을 이용한 입력 가드레일

우리의 에이전트 팀은 이제 선호도를 기억하고 도구를 효과적으로 사용하는 등 더욱 똑똑해지고 있습니다. 하지만 실제 환경에서는 **문제가 될 수 있는 요청이 LLM에 도달하기 전에** 이를 제어할 수 있는 안전 장치가 필요합니다.

ADK는 이러한 목적을 위해 **콜백(Callback)** 기능을 제공합니다. 특히 `before_model_callback`은 **입력 안전성 확보**에 매우 유용합니다.

**`before_model_callback`이란?**

* 에이전트가 대화 기록, 지시사항, 최신 사용자 메시지를 LLM에 보내기 **직전**에 실행되는 Python 함수입니다.  
* **목적:** 요청을 검사하고 필요 시 수정하거나, 사전 정의된 규칙에 따라 요청을 **완전히 차단**할 수 있습니다.

**일반적인 사용 사례:**

* **입력 검증/필터링:** 사용자 입력에 금지된 내용(PII, 특정 키워드 등)이 있는지 확인  
* **가드레일:** 유해하거나 정책에 위배되는 요청이 LLM에 전달되지 않도록 차단  
* **동적 프롬프트 수정:** 세션 상태에서 정보 등을 추출해 LLM 요청 직전에 반영

**작동 방식:**

1. `callback_context: CallbackContext`와 `llm_request: LlmRequest`를 인자로 받는 함수를 정의합니다.  
   * `callback_context`: 에이전트 정보, 세션 상태(`callback_context.state`) 등에 접근 가능  
   * `llm_request`: LLM으로 보낼 전체 요청 (`contents`, `config`) 포함  
2. 함수 내부에서는 다음을 수행할 수 있습니다:  
   * **검사:** `llm_request.contents`(특히 마지막 사용자 메시지)를 확인  
   * **수정(주의 필요):** `llm_request` 일부를 변경할 수 있음  
   * **차단:** `LlmResponse` 객체를 반환하면 해당 응답이 즉시 사용자에게 반환되고 LLM 호출은 **건너뜀**  
   * **허용:** `None`을 반환하면 ADK는 (수정된) 요청을 LLM에 정상 전달함

**이번 단계에서 할 일:**

1. 특정 키워드("BLOCK")가 포함된 사용자 입력을 차단하는 `before_model_callback` 함수(`block_keyword_guardrail`) 정의  
2. 이 콜백을 `Step 4`의 상태 유지 루트 에이전트(`weather_agent_v4_stateful`)에 추가  
3. 동일한 상태 유지 세션 서비스를 사용하여 새로운 실행기(runner)를 생성해 상태를 유지  
4. 일반 입력과 차단 키워드가 포함된 입력을 전송하여 가드레일 동작 테스트

---

**1. 가드레일 콜백 함수 정의하기**

이 함수는 `llm_request`의 내용 중 마지막 사용자 메시지를 검사합니다.  
만약 "BLOCK"이라는 단어가 **대소문자 구분 없이** 포함되어 있다면, 흐름을 차단하기 위해 `LlmResponse` 객체를 생성하여 반환합니다.  
그렇지 않으면 `None`을 반환하여 LLM 호출이 정상적으로 진행되도록 합니다.

In [ ]:
# @title 1. before_model_callback 가드레일 정의하기

# 필요한 import가 준비되어 있는지 확인
from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse
from google.genai import types # 응답을 생성하기 위해 필요
from typing import Optional

def block_keyword_guardrail(
    callback_context: CallbackContext, llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """
    Inspects the latest user message for 'BLOCK'. If found, blocks the LLM call
    and returns a predefined LlmResponse. Otherwise, returns None to proceed.
    """
    agent_name = callback_context.agent_name # 모델 호출이 가로채진 에이전트의 이름을 가져옵니다
    print(f"--- Callback: block_keyword_guardrail running for agent: {agent_name} ---")

    # 요청 기록에서 최신 사용자 메시지의 텍스트를 추출합니다
    last_user_message_text = ""
    if llm_request.contents:
        # 역할이 'user'인 가장 최근 메시지를 찾습니다
        for content in reversed(llm_request.contents):
            if content.role == 'user' and content.parts:
                # 간단하게 처리하기 위해 텍스트가 첫 번째 부분에 있다고 가정합니다
                if content.parts[0].text:
                    last_user_message_text = content.parts[0].text
                    break # 마지막 사용자 메시지 텍스트를 찾았습니다

    print(f"--- Callback: Inspecting last user message: '{last_user_message_text[:100]}...' ---") # 첫 100개 문제 출력

    # --- 가드레일 로직 ---
    keyword_to_block = "BLOCK"
    if keyword_to_block in last_user_message_text.upper(): # 대소문자 구분 없이 체크
        print(f"--- Callback: Found '{keyword_to_block}'. Blocking LLM call! ---")
        # 선택적으로, 차단 이벤트를 기록하기 위해 상태에 플래그를 설정할 수 있습니다.
        callback_context.state["guardrail_block_keyword_triggered"] = True
        print(f"--- Callback: Set state 'guardrail_block_keyword_triggered': True ---")

        # 흐름을 중단하고 이 응답을 대신 보내기 위해 LlmResponse를 생성하여 반환합니다
        return LlmResponse(
            content=types.Content(
                role="model", # 에이전트의 관점에서 보낸 응답처럼 가장합니다
                parts=[types.Part(text=f"I cannot process this request because it contains the blocked keyword '{keyword_to_block}'.")],
            )
            # 참고: 필요하다면 여기에서 error_message 필드를 설정할 수도 있습니다
        )
    else:
        # 키워드가 발견되지 않았으므로 요청이 LLM으로 전달되도록 허용합니다
        print(f"--- Callback: Keyword not found. Allowing LLM call for {agent_name}. ---")
        return None # None을 반환하면 ADK가 정상적으로 계속 진행하라는 신호를 보냅니다

print("✅ block_keyword_guardrail function defined.")


---

**2. 루트 에이전트를 콜백을 사용하도록 업데이트하기**

새로운 가드레일 함수를 사용하도록 `before_model_callback` 매개변수를 추가하여 루트 에이전트를 재정의합니다. 구분을 위해 새로운 버전 이름을 부여합니다.

*중요:* 이전 단계에서 정의된 구성 요소가 현재 컨텍스트에 없을 경우,  
서브 에이전트(`greeting_agent`, `farewell_agent`)와 상태 유지 도구(`get_weather_stateful`)도 함께 재정의해야 합니다.  
이는 루트 에이전트 정의 시 모든 구성 요소에 접근할 수 있도록 하기 위함입니다.

In [ ]:
# @title 2. `before_model_callback`을 포함하여 루트 에이전트를 업데이트합니다


# --- 하위 에이전트를 재정의합니다 (이 컨텍스트에서 존재하도록 보장하기 위함입니다) ---
greeting_agent = None
try:
    # 정의된 모델 상수 사용
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent", # 일관성을 위해 원래 이름을 유지합니다
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Sub-Agent '{greeting_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Greeting agent. Check Model/API Key ({MODEL_GPT_4O}). Error: {e}")

farewell_agent = None
try:
    # 정의된 모델 상수 사용
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent", # 원래 이름을 유지합니다
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye' tool. Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Sub-Agent '{farewell_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Farewell agent. Check Model/API Key ({MODEL_GPT_4O}). Error: {e}")


# --- 콜백을 포함하여 루트 에이전트를 정의합니다 ---
root_agent_model_guardrail = None
runner_root_model_guardrail = None

# 진행하기 전에 모든 구성 요소를 확인하세요
if greeting_agent and farewell_agent and 'get_weather_stateful' in globals() and 'block_keyword_guardrail' in globals():

    # `MODEL_GEMINI_2_5_PRO`와 같은 정의된 모델 상수를 사용하세요
    root_agent_model = MODEL_GEMINI_2_0_FLASH

    root_agent_model_guardrail = Agent(
        name="weather_agent_v5_model_guardrail", # 명확성을 위해 새로운 버전 이름을 사용합니다
        model=root_agent_model,
        description="Main agent: Handles weather, delegates greetings/farewells, includes input keyword guardrail.",
        instruction="You are the main Weather Agent. Provide weather using 'get_weather_stateful'. "
                    "Delegate simple greetings to 'greeting_agent' and farewells to 'farewell_agent'. "
                    "Handle only weather requests, greetings, and farewells.",
        tools=[get_weather],
        sub_agents=[greeting_agent, farewell_agent], # 재정의된 하위 에이전트를 참조합니다
        output_key="last_weather_report", # Step 4의 output_key 유지
        before_model_callback=block_keyword_guardrail # <<< 가드레일 콜백을 할당합니다
    )
    print(f"✅ Root Agent '{root_agent_model_guardrail.name}' created with before_model_callback.")

    # --- 이 에이전트를 위한 Runner 생성, 동일한 상태 유지 세션 서비스 사용 ---
    # Step 4에서 정의된 session_service_stateful이 존재하는지 확인하세요
    if 'session_service_stateful' in globals():
        runner_root_model_guardrail = Runner(
            agent=root_agent_model_guardrail,
            app_name=APP_NAME, # Use consistent APP_NAME
            session_service=session_service_stateful # <<< Step 4의 서비스 사용
        )
        print(f"✅ Runner created for guardrail agent '{runner_root_model_guardrail.agent.name}', using stateful session service.")
    else:
        print("❌ Cannot create runner. 'session_service_stateful' from Step 4 is missing.")

else:
    print("❌ Cannot create root agent with model guardrail. One or more prerequisites are missing or failed initialization:")
    if not greeting_agent: print("   - Greeting Agent")
    if not farewell_agent: print("   - Farewell Agent")
    if 'get_weather_stateful' not in globals(): print("   - 'get_weather_stateful' tool")
    if 'block_keyword_guardrail' not in globals(): print("   - 'block_keyword_guardrail' callback")

---

**3. 가드레일 동작 테스트**

이제 가드레일의 동작을 테스트해봅니다.  
*Step 4에서 사용한 동일한 세션* (`SESSION_ID_STATEFUL`)을 사용하여 상태가 변경 이후에도 유지됨을 확인합니다.

1. 일반적인 날씨 요청을 전송합니다 (가드레일을 통과하여 정상 실행되어야 함).  
2. "BLOCK"이 포함된 요청을 전송합니다 (콜백에 의해 차단되어야 함).  
3. 인사 메시지를 전송합니다 (루트 에이전트의 가드레일을 통과하고 위임되어 정상 실행되어야 함).

In [ ]:
# @title 3. 모델 입력 가드레일 테스트를 위한 상호작용

# 가드레일 에이전트를 위한 실행기가 준비되어 있는지 확인하세요
if runner_root_model_guardrail:
  async def run_guardrail_test_conversation():
      print("\n--- Testing Model Input Guardrail ---")

      # 콜백이 포함된 에이전트와 기존 상태 유지 세션 ID를 사용하는 실행 함수
      interaction_func = lambda query: call_agent_async(query,
      runner_root_model_guardrail, USER_ID_STATEFUL, SESSION_ID_STATEFUL # <-- 올바른 ID 전달
  )
      # 1. 일반 요청 (콜백 통과, Step 4에서 변경된 상태로 화씨 사용)
      await interaction_func("What is the weather in London?")

      # 2. 차단 키워드가 포함된 요청
      await interaction_func("BLOCK the request for weather in Tokyo")

      # 3. 일반 인사 (루트 에이전트 콜백 통과, 위임 정상 동작)
      await interaction_func("Hello again")


  # 대화 실행
  await run_guardrail_test_conversation()

  # 선택 사항: 콜백에 의해 설정된 트리거 플래그가 상태에 있는지 확인
  final_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                       user_id=USER_ID_STATEFUL,
                                                       session_id=SESSION_ID_STATEFUL)
  if final_session:
      print("\n--- Final Session State (After Guardrail Test) ---")
      print(f"Guardrail Triggered Flag: {final_session.state.get('guardrail_block_keyword_triggered')}")
      print(f"Last Weather Report: {final_session.state.get('last_weather_report')}") # Should be London weather
      print(f"Temperature Unit: {final_session.state.get('user_preference_temperature_unit')}") # Should be Fahrenheit
  else:
      print("\n❌ Error: Could not retrieve final session state.")

else:
  print("\n⚠️ Skipping model guardrail test. Runner ('runner_root_model_guardrail') is not available.")



---

실행 흐름을 관찰해보면 다음과 같습니다:

1. **런던 날씨:** `weather_agent_v5_model_guardrail`에 대해 콜백이 실행되어 메시지를 검사하고, `"Keyword not found. Allowing LLM call."`을 출력한 뒤 `None`을 반환합니다. 에이전트는 정상적으로 진행되어 `get_weather_stateful` 도구를 호출하며, Step 4에서 변경된 상태에 따라 "화씨" 단위를 사용합니다. 이 응답은 `output_key`를 통해 `last_weather_report` 상태를 업데이트합니다.

2. **BLOCK 요청:** 콜백이 다시 실행되어 메시지를 검사하고 "BLOCK" 키워드를 발견합니다. `"Blocking LLM call!"`을 출력하고 상태에 플래그를 설정한 후, 사전 정의된 `LlmResponse`를 반환합니다. 이 턴에서는 LLM 호출이 *전혀 이루어지지 않으며*, 사용자에게는 콜백이 반환한 차단 메시지가 표시됩니다.

3. **다시 인사:** 콜백이 실행되어 요청을 허용합니다. 루트 에이전트는 `greeting_agent`로 위임하고, *참고: 루트 에이전트에 정의된 `before_model_callback`은 서브 에이전트에는 자동 적용되지 않습니다.*  
   `greeting_agent`는 정상적으로 진행되어 `say_hello` 도구를 호출하고 인사말을 반환합니다.

당신은 입력 안전 계층을 성공적으로 구현했습니다!  
`before_model_callback`은 **비용이 많이 들거나 위험할 수 있는 LLM 호출 이전에** 규칙을 강제하고 에이전트의 동작을 제어할 수 있는 강력한 메커니즘을 제공합니다.

다음 단계에서는 유사한 개념을 사용하여 **도구(tool) 사용 자체에도 가드레일을 추가**해보겠습니다.

## Step 6: 안전 장치 추가 – 도구 인자 가드레일 (`before_tool_callback`)

5단계에서는 LLM에 도달하기 *이전*에 사용자 입력을 검사하고 차단할 수 있는 가드레일을 추가했습니다.  
이번에는 LLM이 도구 사용을 결정한 *이후*이지만 도구가 실제로 실행되기 *이전*에 제어할 수 있는 또 다른 계층을 추가합니다.  
이는 LLM이 도구에 전달하려는 *인자(arguments)*를 검증하는 데 유용합니다.

ADK는 이러한 목적을 위해 `before_tool_callback`을 제공합니다.

**`before_tool_callback`이란?**

* LLM이 도구 사용을 요청하고 인자를 결정한 후, 해당 도구 함수가 실행되기 *직전에* 호출되는 Python 함수입니다.  
* **목적:** 도구 인자 검증, 특정 입력 기반 도구 실행 차단, 인자의 동적 수정, 리소스 사용 정책 적용

**일반적인 사용 사례:**

* **인자 검증:** LLM이 제공한 인자가 유효한지, 허용된 범위 내에 있는지, 기대하는 형식인지 확인  
* **리소스 보호:** 특정 파라미터로 인해 비용이 크거나 제한된 데이터를 접근하려는 경우 도구 호출 차단  
* **인자 동적 수정:** 세션 상태나 기타 맥락 정보를 기반으로 도구 실행 전에 인자를 조정

**작동 방식:**

1. `tool: BaseTool`, `args: Dict[str, Any]`, `tool_context: ToolContext`를 인자로 받는 함수를 정의합니다.  
   * `tool`: 호출될 도구 객체 (`tool.name`으로 이름 확인 가능)  
   * `args`: LLM이 생성한 도구 호출 인자 딕셔너리  
   * `tool_context`: 세션 상태(`tool_context.state`), 에이전트 정보 등 접근 가능  
2. 함수 내부에서는 다음 작업을 수행할 수 있습니다:  
   * **검사:** `tool.name`과 `args` 딕셔너리를 확인  
   * **수정:** `args` 딕셔너리 내 값을 *직접* 수정 가능. `None`을 반환하면 수정된 인자로 도구가 실행됩니다.  
   * **차단/오버라이드 (가드레일):** **딕셔너리**를 반환하면 ADK는 이를 도구 호출 결과로 처리하며, 원래 도구 함수는 *실행되지 않습니다*. 이 딕셔너리는 해당 도구의 예상 반환 형식과 일치해야 합니다.  
   * **허용:** `None`을 반환하면 도구가 (수정된 인자가 있다면 그것으로) 정상 실행됩니다.

**이번 단계에서는 다음을 수행합니다:**

1. `before_tool_callback` 함수(`block_paris_tool_guardrail`)를 정의하여 `get_weather_stateful` 도구가 "Paris"를 인자로 받을 경우 감지  
2. "Paris"가 감지되면 도구를 차단하고 사용자 정의 오류 딕셔너리를 반환  
3. 루트 에이전트(`weather_agent_v6_tool_guardrail`)를 업데이트하여  
   * `before_model_callback`과  
   * 새로 만든 `before_tool_callback`을 *모두* 포함  
4. 동일한 상태 유지 세션 서비스를 사용하는 새로운 실행기를 생성  
5. 허용된 도시들과 차단된 도시("Paris")에 대해 날씨 요청을 전송하여 흐름을 테스트

---

**1. 도구 가드레일 콜백 함수 정의**

이 함수는 `get_weather_stateful` 도구를 대상으로 합니다.  
`city` 인자를 검사하며, 값이 "Paris"일 경우 해당 도구의 오류 응답 형식과 유사한 오류 딕셔너리를 반환합니다.  
그 외의 경우에는 `None`을 반환하여 도구가 정상 실행되도록 허용합니다.

In [ ]:
# @title 1. before_tool_callback 가드레일 정의하기

# 필요한 import가 준비되어 있는지 확인하세요
from google.adk.tools.base_tool import BaseTool
from google.adk.tools.tool_context import ToolContext
from typing import Optional, Dict, Any # 타입 힌팅

def block_paris_tool_guardrail(
    tool: BaseTool, args: Dict[str, Any], tool_context: ToolContext
) -> Optional[Dict]:
    """
    Checks if 'get_weather_stateful' is called for 'Paris'.
    If so, blocks the tool execution and returns a specific error dictionary.
    Otherwise, allows the tool call to proceed by returning None.
    """
    tool_name = tool.name
    agent_name = tool_context.agent_name # 도구 호출을 시도하는 에이전트
    print(f"--- Callback: block_paris_tool_guardrail running for tool '{tool_name}' in agent '{agent_name}' ---")
    print(f"--- Callback: Inspecting args: {args} ---")

    # --- 가드레일 로직 ---
    target_tool_name = "get_weather_stateful" # FunctionTool에서 사용하는 함수 이름과 일치시킵니다
    blocked_city = "paris"

    # 올바른 도구인지, 그리고 `city` 인자가 차단 대상 도시와 일치하는지 확인합니다
    if tool_name == target_tool_name:
        city_argument = args.get("city", "") # 안전하게 `'city'` 인자를 가져옵니다
        if city_argument and city_argument.lower() == blocked_city:
            print(f"--- Callback: Detected blocked city '{city_argument}'. Blocking tool execution! ---")
            # 선택적으로 상태를 업데이트합니다
            tool_context.state["guardrail_tool_block_triggered"] = True
            print(f"--- Callback: Set state 'guardrail_tool_block_triggered': True ---")

            # 오류에 대한 도구의 예상 출력 형식과 일치하는 딕셔너리를 반환합니다
            # 이 딕셔너리는 도구의 결과로 처리되며, 실제 도구 실행은 건너뜁니다
            return {
                "status": "error",
                "error_message": f"Policy restriction: Weather checks for '{city_argument.capitalize()}' are currently disabled by a tool guardrail."
            }
        else:
             print(f"--- Callback: City '{city_argument}' is allowed for tool '{tool_name}'. ---")
    else:
        print(f"--- Callback: Tool '{tool_name}' is not the target tool. Allowing. ---")


    # 위의 검사에서 딕셔너리를 반환하지 않았다면, 도구 실행을 허용합니다
    print(f"--- Callback: Allowing tool '{tool_name}' to proceed. ---")
    return None # `None`을 반환하면 실제 도구 함수가 실행됩니다

print("✅ block_paris_tool_guardrail function defined.")



---

**2. 루트 에이전트를 두 콜백 모두 사용하도록 업데이트하기**

이번에는 루트 에이전트(`weather_agent_v6_tool_guardrail`)를 다시 정의하되, Step 5에서 사용한 `before_model_callback`에 더해 `before_tool_callback` 파라미터도 함께 추가합니다.

*단독 실행 시 주의:* Step 5와 마찬가지로, 이 에이전트를 정의하기 전에 모든 전제 조건(서브 에이전트, 도구, `before_model_callback`)이 실행 컨텍스트 내에 정의되어 있어야 합니다.

In [ ]:
# @title 2. 두 콜백을 모두 포함하여 루트 에이전트 업데이트 (단독 실행 가능하게)

# --- 전제 조건이 정의되어 있는지 확인 ---
# (다음 항목들이 정의되었거나 실행되었는지 확인: Agent, LiteLlm, Runner, ToolContext,
#  MODEL 상수들, say_hello, say_goodbye, greeting_agent, farewell_agent,
#  get_weather_stateful, block_keyword_guardrail, block_paris_tool_guardrail)

# --- 서브 에이전트를 재정의 (현재 컨텍스트에 존재하도록 보장) ---
greeting_agent = None
try:
    # MODEL_GPT_4O와 같은 정의된 모델 상수를 사용합니다
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent", # 일관성을 위해 기존 이름 사용
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Sub-Agent '{greeting_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Greeting agent. Check Model/API Key ({MODEL_GPT_4O}). Error: {e}")

farewell_agent = None
try:
    # MODEL_GPT_4O와 같은 정의된 모델 상수를 사용합니다
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent", # 기존 이름 사용
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye' tool. Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Sub-Agent '{farewell_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Farewell agent. Check Model/API Key ({MODEL_GPT_4O}). Error: {e}")

# --- 두 콜백을 모두 가진 루트 에이전트 정의 ---
root_agent_tool_guardrail = None
runner_root_tool_guardrail = None

if ('greeting_agent' in globals() and greeting_agent and
    'farewell_agent' in globals() and farewell_agent and
    'get_weather_stateful' in globals() and
    'block_keyword_guardrail' in globals() and
    'block_paris_tool_guardrail' in globals()):

    root_agent_model = MODEL_GEMINI_2_0_FLASH

    root_agent_tool_guardrail = Agent(
        name="weather_agent_v6_tool_guardrail", # 새 버전 이름
        model=root_agent_model,
        description="Main agent: Handles weather, delegates, includes input AND tool guardrails.",
        instruction="You are the main Weather Agent. Provide weather using 'get_weather_stateful'. "
                    "Delegate greetings to 'greeting_agent' and farewells to 'farewell_agent'. "
                    "Handle only weather, greetings, and farewells.",
        tools=[get_weather_stateful],
        sub_agents=[greeting_agent, farewell_agent],
        output_key="last_weather_report",
        before_model_callback=block_keyword_guardrail, # 모델 가드레일 유지
        before_tool_callback=block_paris_tool_guardrail # <<< 도구 가드레일 추가
    )
    print(f"✅ Root Agent '{root_agent_tool_guardrail.name}' created with BOTH callbacks.")

    # --- Runner 생성, 동일한 Stateful 세션 서비스 사용 ---
    if 'session_service_stateful' in globals():
        runner_root_tool_guardrail = Runner(
            agent=root_agent_tool_guardrail,
            app_name=APP_NAME,
            session_service=session_service_stateful # <<< Step 4/5에서 사용한 서비스 그대로 사용
        )
        print(f"✅ Runner created for tool guardrail agent '{runner_root_tool_guardrail.agent.name}', using stateful session service.")
    else:
        print("❌ Cannot create runner. 'session_service_stateful' from Step 4/5 is missing.")

else:
    print("❌ Cannot create root agent with tool guardrail. Prerequisites missing.")



---

**3. 도구 가드레일 테스트를 위한 상호작용**

이전 단계들과 동일하게 상태 유지 세션(`SESSION_ID_STATEFUL`)을 사용하여 상호작용 흐름을 테스트합니다.

1. "New York"의 날씨 요청: 두 콜백을 모두 통과하고, 도구가 실행됨 (상태에 저장된 화씨 선호 사용)  
2. "Paris"의 날씨 요청: `before_model_callback`을 통과함. LLM이 `get_weather_stateful(city='Paris')` 호출을 결정함. `before_tool_callback`이 이를 가로채 차단하고 오류 딕셔너리를 반환함. 에이전트는 이 오류를 그대로 전달  
3. "London"의 날씨 요청: 두 콜백을 모두 통과하고 도구가 정상 실행됨

In [ ]:
# @title 3. 도구 인자 가드레일 테스트를 위한 상호작용

# 도구 가드레일 에이전트를 위한 실행기가 준비되어 있는지 확인하세요
if runner_root_tool_guardrail:
  async def run_tool_guardrail_test():
      print("\n--- Testing Tool Argument Guardrail ('Paris' blocked) ---")

      # 두 콜백이 모두 포함된 에이전트와 기존 상태 유지 세션을 사용하는 Runner를 사용합니다
      interaction_func = lambda query: call_agent_async(query,
      runner_root_tool_guardrail, USER_ID_STATEFUL, SESSION_ID_STATEFUL
  )
      # 1. 허용된 도시 (두 콜백 모두 통과, 화씨를 사용해야 함)
      await interaction_func("What's the weather in New York?")

      # 2. 막혀있는 도시 (모델 콜백은 통과, 하지만 도구 콜백에 의해 막혀야 함)
      await interaction_func("How about Paris?")

      # 3. 허용된 도시 (다시 잘 동작해야 함)
      await interaction_func("Tell me the weather in London.")

  # 대화 실행
  await run_tool_guardrail_test()

  # 선택 사항: 도구 차단 트리거 플래그가 상태에 설정되었는지 확인합니다
  final_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                       user_id=USER_ID_STATEFUL,
                                                       session_id= SESSION_ID_STATEFUL)
  if final_session:
      print("\n--- Final Session State (After Tool Guardrail Test) ---")
      print(f"Tool Guardrail Triggered Flag: {final_session.state.get('guardrail_tool_block_triggered')}")
      print(f"Last Weather Report: {final_session.state.get('last_weather_report')}") # 런던 날씨여야 함
      print(f"Temperature Unit: {final_session.state.get('user_preference_temperature_unit')}") # 화씨여야 함
  else:
      print("\n❌ Error: Could not retrieve final session state.")

else:
  print("\n⚠️ Skipping tool guardrail test. Runner ('runner_root_tool_guardrail') is not available.")

---

출력 결과 분석:

1. **New York:** `before_model_callback`이 요청을 허용합니다. LLM은 `get_weather_stateful` 호출을 요청합니다. `before_tool_callback`이 실행되어 인자 `{'city': 'New York'}`를 검사하고, "Paris"가 아님을 확인한 뒤 `"Allowing tool..."`을 출력하고 `None`을 반환합니다. 실제 `get_weather_stateful` 함수가 실행되어 상태에서 "Fahrenheit"를 읽고 날씨 보고서를 반환합니다. 이 응답은 에이전트를 통해 전달되며 `output_key`를 통해 상태에 저장됩니다.  

2. **Paris:** `before_model_callback`이 요청을 허용합니다. LLM은 `get_weather_stateful(city='Paris')` 호출을 요청합니다. `before_tool_callback`이 실행되어 인자를 검사하고 "Paris"를 감지한 뒤 `"Blocking tool execution!"`을 출력하고, 상태에 플래그를 설정하며 오류 딕셔너리 `{'status': 'error', 'error_message': 'Policy restriction...'}`를 반환합니다. 실제 `get_weather_stateful` 함수는 **절대 실행되지 않으며**, 에이전트는 이 오류 딕셔너리를 *도구의 출력인 것처럼* 받아 적절한 오류 메시지를 사용자에게 전달합니다.  

3. **London:** New York과 동일하게 동작하며, 두 콜백을 모두 통과해 도구가 성공적으로 실행됩니다. 새로운 London 날씨 보고서는 상태 내 `last_weather_report`를 덮어씁니다.

이제 LLM에 **무엇이 도달하는지**뿐 아니라, LLM이 생성한 **도구 인자에 따라 도구가 어떻게 실행되는지도** 제어할 수 있는 중요한 안전 계층이 추가되었습니다.  
`before_model_callback`과 `before_tool_callback` 같은 콜백은 견고하고 안전하며 정책을 준수하는 에이전트 애플리케이션을 구축하는 데 필수적인 도구입니다.

---

## 결론: 당신의 에이전트 팀이 준비되었습니다!

축하합니다!  
당신은 단순한 날씨 에이전트 하나에서 출발해, ADK(Agent Development Kit)를 사용해 정교한 **멀티 에이전트 시스템**을 성공적으로 구축했습니다.

**지금까지 구현한 내용을 정리해보면 다음과 같습니다:**

- **기본적인 에이전트**와 단일 도구(`get_weather`)로 시작했습니다.  
- LiteLLM을 통해 **여러 LLM 모델(Gemini, GPT-4o, Claude 등)**을 유연하게 활용하는 방법을 익혔습니다.  
- `greeting_agent`, `farewell_agent` 같은 **서브 에이전트**를 만들어 루트 에이전트로부터 **자동 위임**이 가능하도록 모듈화를 적용했습니다.  
- **Session State**를 통해 에이전트에 **기억력**을 부여하고, 사용자 선호도(`temperature_unit`) 및 최근 응답(`output_key`)을 상태에 저장했습니다.  
- `before_model_callback`과 `before_tool_callback`을 통해 각각 **입력 필터링**과 **도구 호출 제한**을 적용하며 **안전 가드레일**을 구현했습니다.

**Weather Bot 팀을 단계별로 확장하며 ADK의 핵심 개념을 직접 다뤄보았습니다.**


### 주요 개념 요약

- **에이전트 & 도구**: 기능 정의의 핵심. 명확한 지침과 설명(docstring)이 매우 중요합니다.  
- **러너(Runner) & 세션 서비스(Session Service)**: 실행 엔진 및 대화 상태를 관리하는 메모리 시스템입니다.  
- **위임(Delegation)**: 멀티 에이전트 구조를 설계함으로써 복잡한 작업을 모듈화하고 효율적으로 분담할 수 있습니다. 자동 흐름에 있어 `description`이 핵심입니다.  
- **세션 상태(`ToolContext`, `output_key`)**: 문맥을 고려한, 맞춤형 다회차 대화를 가능하게 해주는 필수 요소입니다.  
- **콜백(`before_model`, `before_tool`)**: 중요한 작업(LLM 호출, 도구 실행) *이전에* 실행되어 안전성, 유효성 검사, 정책 적용, 동적 수정이 가능합니다.  
- **유연성(LiteLlm)**: 다양한 모델 선택이 가능해 성능, 비용, 기능 면에서 최적의 조합을 사용할 수 있습니다.

### 다음 단계는?

Weather Bot 팀은 훌륭한 출발점입니다. 아래와 같은 방향으로 ADK를 더 확장해보세요:

1. **실제 날씨 API 연동**: `mock_weather_db`를 OpenWeatherMap 같은 실제 날씨 API로 대체  
2. **복잡한 세션 상태 저장**: 선호 도시, 알림 설정, 대화 요약 등 다양한 사용자 정보를 상태에 보관  
3. **위임 정교화**: 루트 에이전트 지시문과 서브 에이전트 설명을 다듬어 보다 섬세한 위임 로직 구현 (예: `forecast_agent` 추가)  
4. **고급 콜백 구현**:  
   - `after_model_callback`으로 LLM 응답 후 포맷 재정렬 또는 정제  
   - `after_tool_callback`으로 도구 결과 후처리 또는 로깅  
   - `before_agent_callback`, `after_agent_callback`으로 에이전트 단 진입/종료 로직 추가  
5. **에러 핸들링 강화**: 도구 실행 실패 시 재시도 로직 등 추가  
6. **세션 상태 영속화**: `InMemorySessionService` 대신 Firestore, Cloud SQL 등 DB와 연동 (직접 구현 또는 추후 ADK 연동 필요)  
7. **스트리밍 UI 연동**: FastAPI 등 웹 프레임워크와 연결하여 실시간 챗 UI 구현 (ADK Streaming Quickstart 참고)

**Agent Development Kit(ADK)는 고도화된 LLM 애플리케이션을 구축할 수 있는 강력한 기반입니다.**  
도구, 상태, 위임, 콜백이라는 개념을 완전히 이해하고 활용함으로써, 복잡하고 지능적인 에이전트 시스템도 능숙하게 설계할 수 있습니다.

**즐거운 개발 되세요!**